## Training results
- Machine Learning; predicting cost: MAE 5.2
- Machine Learning; predicting total_secs + num_unq: MAE 5.2

- Deep Learning; predicting cost: MAE 5.1
- Deep Learning; predicting total_secs + num_unq: MAE ?

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Utils.DataLoader import DataLoader
from Utils.DataTransformer import DataTransformer
from Utils.DuckDb import DuckDb
from datetime import datetime
import pandas as pd
from typing import List
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from Utils import PandasNotebookConfigs

In [3]:
PandasNotebookConfigs.config()

Ajustando display.float.format para %.2f


In [4]:
DUCK_DB_UTILS = DuckDb()
DATA_TRANSFORMER = DataTransformer()

In [5]:
TABLE_NAME = 'treated_dataset_2025_03_08_00h03m'
TABLE_NAME = 'treated_dataset_2025_03_10_10h03m'
TABLE_NAME = 'treated_dataset_2025_03_11_08h03m'

In [6]:
df = DUCK_DB_UTILS.load_table(TABLE_NAME)

In [7]:
len(df)

151620

In [8]:
# Treating DF
df.isna().sum().T

cost                                      0
msno                                      0
safra                                     0
num_25                                    0
num_50                                    0
num_75                                    0
num_985                                   0
num_100                                   0
num_unq                                   0
total_secs                                0
total_hours                               0
msno_1                                    0
payment_method_id                         0
payment_plan_days                         0
plan_list_price                           0
actual_amount_paid                        0
is_auto_renew                             0
is_cancel                                 0
safra_1                                   0
transaction_date_year                     0
transaction_date_month                    0
transaction_date_day                      0
transaction_date_day_of_week    

In [9]:
# Dropping NA values
df = df.dropna(subset=['cost+1M'])

In [10]:
df = df.drop('total_hours', axis=1)

In [11]:
# Feature engineering
df['safra_year'] = df['safra'].astype(str).str[:4].astype(float)
df['safra_month'] = df['safra'].astype(str).str[-2:].astype(float)

In [12]:
    # # Selecting features after Lasso
    # lasso_selected_features = [
    #     # X features
    #     'cost', 'safra', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100',
    #     'num_unq', 'total_secs', 'payment_method_id', 'payment_plan_days', 'plan_list_price',
    #     'is_auto_renew', 'transaction_date_month', 'transaction_date_day', 'transaction_date_day_of_week',
    #     'transaction_date_day_of_year', 'membership_expire_date_month', 'membership_expire_date_day',
    #     'membership_expire_date_day_of_week', 'membership_expire_date_day_of_year', 'discount', 'price_per_month',
    #     'city', 'registered_via', 'registration_init_time_year', 'registration_init_time_month',
    #     'registration_init_time_day', 'registration_init_time_day_of_week', 'registration_init_time_day_of_year',
    #     'cost-2M', 'cost-1M', 'num_unq-2M', 'num_unq-1M', 'total_secs-2M', 'total_secs-1M',

    #     # y targets
    #     'total_secs+1M', 'num_unq+1M', 'cost+1M'
    # ]

    # df = df[lasso_selected_features]

In [13]:
df.isna().sum().T

cost                                  0
msno                                  0
safra                                 0
num_25                                0
num_50                                0
num_75                                0
num_985                               0
num_100                               0
num_unq                               0
total_secs                            0
msno_1                                0
payment_method_id                     0
payment_plan_days                     0
plan_list_price                       0
actual_amount_paid                    0
is_auto_renew                         0
is_cancel                             0
safra_1                               0
transaction_date_year                 0
transaction_date_month                0
transaction_date_day                  0
transaction_date_day_of_week          0
transaction_date_day_of_year          0
membership_expire_date_year           0
membership_expire_date_month          0


In [14]:
df.head(100)

,cost,msno,safra,num_25,num_50,num_75,num_985,num_100,num_unq,total_secs,...,cost-1M,cost+1M,num_unq-2M,num_unq-1M,num_unq+1M,total_secs-2M,total_secs-1M,total_secs+1M,safra_year,safra_month
6,70.95,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,201609,53,28,18,19,665,707,173410,...,72.33,73.11,513.00,747.00,791.00,120546.00,185178.00,190726.00,2016.00,9.00
7,73.11,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,201610,32,12,14,20,755,791,190726,...,70.95,66.11,747.00,707.00,545.00,185178.00,173410.00,133337.00,2016.00,10.00
8,66.11,+++l/EXNMLTijfLBa8p2TUVVVp2aFGSuUI/h7mLmthw=,201611,59,23,11,21,520,545,133337,...,73.11,68.95,707.00,791.00,676.00,173410.00,190726.00,154978.00,2016.00,11.00
12,67.73,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,201603,146,31,28,20,546,598,146757,...,70.49,61.64,845.00,840.00,440.00,217292.00,162097.00,93939.00,2016.00,3.00
13,61.64,+++snpr7pmobhLKUgSHTv/mpkqgBT0tQJ0zQj6qKrqc=,201604,131,23,23,21,334,440,93939,...,67.73,80.63,840.00,598.00,1088.00,162097.00,146757.00,250853.00,2016.00,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,62.65,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,201607,201,44,31,21,346,574,97190,...,56.53,66.28,421.00,368.00,607.00,55928.00,46503.00,131831.00,2016.00,7.00
167,66.28,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,201608,117,33,26,20,464,607,131831,...,62.65,72.89,368.00,574.00,952.00,46503.00,97190.00,180315.00,2016.00,8.00
168,72.89,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,201609,298,80,56,32,584,952,180315,...,66.28,66.43,574.00,607.00,670.00,97190.00,131831.00,130156.00,2016.00,9.00
169,66.43,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,201610,204,42,30,17,443,670,130156,...,72.89,65.96,607.00,952.00,605.00,131831.00,180315.00,128718.00,2016.00,10.00


In [15]:
# Dropping duplicated columns or useless columns
drop_columns = ['msno', 'safra_1', 'safra_2', 'msno_1', 'msno_2']
for col in drop_columns:
    if col not in df.columns:
        print(f'Coluna {col} não existente no DataFrame')
    else:
        print(f'Dropando coluna {col}')
        df = df.drop(col, axis=1,)

Dropando coluna msno
Dropando coluna safra_1
Dropando coluna safra_2
Dropando coluna msno_1
Dropando coluna msno_2


In [16]:
df.head().T

,6,7,8,12,13
cost,70.95,73.11,66.11,67.73,61.64
safra,201609,201610,201611,201603,201604
num_25,53,32,59,146,131
num_50,28,12,23,31,23
num_75,18,14,11,28,23
num_985,19,20,21,20,21
num_100,665,755,520,546,334
num_unq,707,791,545,598,440
total_secs,173410,190726,133337,146757,93939
payment_method_id,39,39,39,41,41


In [17]:
# Turning columns into categories
cat_columns = [
    'payment_method_id',
    'city',
    'registered_via',
]

df = DATA_TRANSFORMER.convert_to_category(
    df,
    cat_columns
)

df = pd.get_dummies(df, columns=cat_columns)

In [18]:
df.dtypes

cost                 float64
safra                  int64
num_25                 int64
num_50                 int64
num_75                 int64
                      ...   
registered_via_13       bool
registered_via_3        bool
registered_via_4        bool
registered_via_7        bool
registered_via_9        bool
Length: 91, dtype: object

In [19]:
# target_cols = 'cost+1M'

# X = df.drop(target_cols, axis=1)
# y = df[target_cols]

# X.drop('total_secs+1M', axis=1, inplace=True)
# X.drop('num_unq+1M', axis=1, inplace=True)

target_cols = ['total_secs+1M', 'num_unq+1M']
X = df.drop(target_cols, axis=1)
y = df[target_cols]

In [20]:
def get_future_data_columns(df: pd.DataFrame) -> List[str]:
    result = []
    for col in df.columns:
        if '+1M' in col:
            result.append(col)

    return result

In [21]:
# Removing all future data (M+1) from the data set
def remove_future_data_columns(df: pd.DataFrame) -> pd.DataFrame:
    to_remove = get_future_data_columns(df)
    df.drop(to_remove, axis=1, inplace=True)
    return df

In [22]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components=7)
# X = pca.fit_transform(X)

In [23]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

cols_to_remove = get_future_data_columns(X)
print(f'Necessário remover colunas {cols_to_remove}')

# Split into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

X_train = remove_future_data_columns(X_train)
X_val = remove_future_data_columns(X_val)

# Standardize numerical features (use the same scaler for test data)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test.drop(cols_to_remove, axis=1)) # Removing columns from test only here, cause we'll need it later!

2025-03-11 08:33:24.751676: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-11 08:33:24.806620: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Necessário remover colunas ['cost+1M']


In [24]:
X_val.columns

Index(['cost', 'safra', 'num_25', 'num_50', 'num_75', 'num_985', 'num_100',
       'num_unq', 'total_secs', 'payment_plan_days', 'plan_list_price',
       'actual_amount_paid', 'is_auto_renew', 'is_cancel',
       'transaction_date_year', 'transaction_date_month',
       'transaction_date_day', 'transaction_date_day_of_week',
       'transaction_date_day_of_year', 'membership_expire_date_year',
       'membership_expire_date_month', 'membership_expire_date_day',
       'membership_expire_date_day_of_week',
       'membership_expire_date_day_of_year', 'discount', 'price_per_month',
       'is_active', 'registration_init_time_year',
       'registration_init_time_month', 'registration_init_time_day',
       'registration_init_time_day_of_week',
       'registration_init_time_day_of_year', 'cost-2M', 'cost-1M',
       'num_unq-2M', 'num_unq-1M', 'total_secs-2M', 'total_secs-1M',
       'safra_year', 'safra_month', 'payment_method_id_11',
       'payment_method_id_14', 'payment_method_id_1

In [26]:
from tensorflow import keras
from tensorflow.keras import layers

# Define the deep learning model
model = keras.Sequential([
    layers.Dense(512, activation='relu', input_shape=(88,)),
    layers.Dense(256, activation='relu'),  # First hidden layer (more neurons)
    layers.BatchNormalization(),
    layers.Dropout(0.3),  # Drops 30% of neurons randomly

    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.3),

    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(2)  # Output layer for regression
])

# optimizer = keras.optimizers.Adam(learning_rate=0.0005)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Model summary
model.summary()

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=300, batch_size=32, verbose=1,
    callbacks=[early_stopping]
)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               45568     
                                                                 
 dense_7 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_2 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 256)               0         
                                                                 
 dense_8 (Dense)             (None, 128)               32896     
                                                                 
 batch_normalization_3 (Batc  (None, 128)              512       
 hNormalization)                                      

2025-03-11 08:34:27.689988: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1989/1989 [==============================] - 13s 6ms/step - loss: 4571576320.0000 - mae: 31333.5957 - val_loss: 2425902336.0000 - val_mae: 23090.1016
Epoch 2/300
1989/1989 [==============================] - 12s 6ms/step - loss: 2639463936.0000 - mae: 24436.9609 - val_loss: 2346433024.0000 - val_mae: 22882.1465
Epoch 3/300
1989/1989 [==============================] - 12s 6ms/step - loss: 2525882624.0000 - mae: 23880.6426 - val_loss: 2316496640.0000 - val_mae: 22635.5352
Epoch 4/300
1989/1989 [==============================] - 12s 6ms/step - loss: 2482472704.0000 - mae: 23607.4707 - val_loss: 2319711744.0000 - val_mae: 22705.6758
Epoch 5/300
1989/1989 [==============================] - 13s 6ms/step - loss: 2461205504.0000 - mae: 23540.8359 - val_loss: 2336003072.0000 - val_mae: 22686.1426
Epoch 6/300
1989/1989 [==============================] - 13s 6ms/step - loss: 2427561216.0000 - mae: 23326.4434 - val_loss: 2323084544.0000 - val_mae: 22448.9590
Epoch 7/300
1989/1989 [=================

KeyboardInterrupt: 

In [ ]:
model.save('./saved-deep-learning-model.h5')

In [ ]:
preds = model.predict(X_test_scaled)
preds[0], preds[1], preds[2]

In [ ]:
compare_df = X_test.copy()
# compare_df

In [ ]:
compare_df['predicted_total_secs'], compare_df['predicted_num_unq'] = preds[:, 0], preds[:, 1]

In [ ]:
compare_df['total_secs+1M'], compare_df['num_unq+1M'] = y_test['total_secs+1M'], y_test['num_unq+1M']

In [ ]:
compare_df['predicted_cost+1M'] = DATA_TRANSFORMER.create_cost_column(compare_df['predicted_num_unq'], compare_df['predicted_total_secs'])

In [ ]:
compare_df[['total_secs+1M', 'predicted_total_secs', 'num_unq+1M', 'predicted_num_unq']]

In [ ]:
show_scores_2(compare_df['cost+1M'], compare_df['predicted_cost+1M'])

In [ ]:
df['total_secs'].mean(), df['num_unq'].mean(),

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Define the deep learning model
model = keras.Sequential([
    # layers.Dense(512, activation='relu', input_shape=(91,)),
    layers.Dense(256, activation='relu', input_shape=(88,)),  # First hidden layer (more neurons)
    # layers.BatchNormalization(),
    # layers.Dropout(0.3),  # Drops 30% of neurons randomly

    layers.Dense(128, activation='relu'),
    # layers.BatchNormalization(),
    # layers.Dropout(0.3),

    layers.Dense(64, activation='relu'),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer for regression
])

# optimizer = keras.optimizers.Adam(learning_rate=0.0005)

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model summary
model.summary()

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100, batch_size=32, verbose=1,
    # callbacks=[early_stopping]
)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

# Define the deep learning model
model = keras.Sequential([
    # layers.Dense(512, activation='relu', input_shape=(43,)),
    layers.Dense(256, activation='relu', input_shape=(7,)),  # First hidden layer (more neurons)
    # layers.BatchNormalization(),
    # layers.Dropout(0.3),  # Drops 30% of neurons randomly

    layers.Dense(128, activation='relu'),
    # layers.BatchNormalization(),
    # layers.Dropout(0.3),

    layers.Dense(64, activation='relu'),
    # layers.Dense(32, activation='relu'),
    layers.Dense(1)  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Model summary
model.summary()

# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_data=(X_val_scaled, y_val),
    epochs=100, batch_size=32, verbose=1,
    callbacks=[early_stopping]
)

In [ ]:
# Evaluate on test data
test_loss, test_mae = model.evaluate(X_test_scaled, y_test)
print(f"Test MAE: {test_mae:.4f}")

# Make predictions
y_pred = model.predict(X_test_scaled)

# Convert predictions into a DataFrame
df_results = pd.DataFrame({"Actual": y_test.values, "Predicted": y_pred.flatten()})
print(df_results.head())

In [ ]:
len(X), len(y)

In [ ]:
X.head()

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# X_train_cost = X_train['cost+1M']
# X_train = X_train.drop('cost+1M', axis=1)

# X_test_cost = X_test['cost+1M']
# X_test = X_test.drop('cost+1M', axis=1)

In [ ]:
len(X_train), len(X_test), len(y_train), len(y_test)

In [ ]:
# model = RandomForestRegressor(
#     n_jobs=-1,
#     random_state=42,
# )

# model.fit(X_train, y_train)

In [ ]:
# Create evaluation function (the competition uses RMLSE)
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score, mean_squared_error
import numpy as np


def rmsle(y_test, y_preds):
    '''
    Calculates root mean squared error between predictions and true labels
    '''
    return str(np.sqrt(mean_squared_log_error(y_test, y_preds)))


def show_scores(model, X_train, X_test, y_train, y_test):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_test)

    # When val_preds score is bigger, it means that your model has overfitting problems
    scores = {
        'Training MAE': mean_absolute_error(y_train, train_preds),
        'Valid MAE': mean_absolute_error(y_test, val_preds),

        # 'Training RMSLE': rmsle(y_train, train_preds),
        # 'Valid RMSLE': rmsle(y_test, val_preds),

        'Training R^2': r2_score(y_train, train_preds),
        'Valid R^2': r2_score(y_test, val_preds),
    }

    return scores


def show_scores_2(y_true, y_preds):
    # When val_preds score is bigger, it means that your model has overfitting problems
    scores = {
        'MAE': mean_absolute_error(y_true, y_preds),
        'MSE': mean_squared_error(y_true, y_preds),
        'R^2': r2_score(y_true, y_preds),
    }

    return scores

In [ ]:
X_train.head().T

In [ ]:
X.head()

In [ ]:
X_test.head()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR

# Initialize the models
models = [
    ('Linear Regression', LinearRegression()),
    ('Decision Tree Regressor', DecisionTreeRegressor(random_state=42)),
    ('Random Forest Regressor', RandomForestRegressor(random_state=42, n_jobs=-1)),
    ('Gradient Boosting Regressor', GradientBoostingRegressor(random_state=42)),
    # ('SVR', SVR())
]

saved_models = {}

# Train and evaluate the models
results = {}
for name, model in models:
    print(f'{datetime.now()} treinando utilizando modelo {model}')
    model.fit(X_train, y_train)
    res = show_scores(model, X_train, X_test, y_train, y_test)
    results[name] = res
    saved_models[name] = model

# Print the results
for name, mse in results.items():
    print(f'{name} MSE: {mse}')

In [ ]:
# Tune Lasso Regression
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# param_grid = {'alpha': np.logspace(-4, 2, 20)}
# lasso = Lasso()
# lasso_cv = GridSearchCV(lasso, param_grid, scoring='neg_mean_absolute_error', cv=5)
# lasso_cv.fit(X_train, y_train)

# print(f"Best alpha for Lasso: {lasso_cv.best_params_}")



# Create a Pipeline that scales data before applying Lasso
pipeline = Pipeline([
    ('scaler', StandardScaler()),  
    ('lasso', Lasso(max_iter=5000))
])

lasso_cv = GridSearchCV(
    pipeline, 
    param_grid={'lasso__alpha': np.logspace(-2, 2, 10)},  # Adjust for pipeline
    scoring='neg_mean_absolute_error',
    cv=5
)
lasso_cv.fit(X_train, y_train)

print(f"Best alpha for Lasso: {lasso_cv.best_params_}")

In [ ]:
import matplotlib.pyplot as plt

selected_features = []

for i in range(0, 2):
    feature_importance = pd.Series(lasso_cv.best_estimator_.named_steps['lasso'].coef_[i], index=X_train.columns)
    features = feature_importance[feature_importance.abs() > 0.01].index.tolist()
    print(features)

    feature_importance[feature_importance.abs() > 0.01].sort_values().plot(kind="barh", figsize=(8, 5))

    for f in features:
        if f not in selected_features:
            selected_features.append(f)

selected_features

In [ ]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import GradientBoostingRegressor

# Define Hyperparameter Search Space
param_dist = {
    'n_estimators': np.arange(100, 1000, 100),  # Number of boosting stages
    'learning_rate': np.linspace(0.01, 0.2, 10),  # Step size
    'max_depth': np.arange(3, 10, 1),  # Tree depth
    'subsample': [0.7, 0.8, 0.9, 1.0],  # Fraction of samples per tree
    'min_samples_split': np.arange(2, 20, 2),  # Min samples required to split
}

# Initialize Gradient Boosting Regressor
gbr = GradientBoostingRegressor(random_state=42)

# Perform Randomized Search
random_search = RandomizedSearchCV(
    estimator=gbr,
    param_distributions=param_dist,
    n_iter=20,  # Number of random combinations to try
    scoring='neg_mean_squared_error',
    cv=5,  # Cross-validation folds
    n_jobs=-1,  # Use all available CPUs
    random_state=42
)

# Fit Randomized Search
random_search.fit(X_train, y_train)

# Get the Best Model
best_gbr = random_search.best_estimator_

# Evaluate Performance
y_pred_best = best_gbr.predict(X_test)
print(f"Best Model MSE: {mean_squared_error(y_test, y_pred_best):.4f}")
print(f"Best Parameters: {random_search.best_params_}")

In [ ]:
choosed_model = saved_models['Decision Tree Regressor']
# choosed_model = ridge_cv
# choosed_model = lasso_cv
choosed_model = saved_models['Linear Regression']
choosed_model = saved_models['Random Forest Regressor']
choosed_model = saved_models['Gradient Boosting Regressor']

In [ ]:
print('Real data:')
print(y_test.iloc[0][target_cols])

test_df_preds = choosed_model.predict(pd.DataFrame(X_test.iloc[0]).T)

print('Predicted: ')
print(test_df_preds)

In [ ]:
preds = choosed_model.predict(X_test)
preds

In [ ]:
compare_df = X_test.copy()

In [ ]:
compare_df['predicted_cost'] = preds
# compare_df['predicted_total_secs'], compare_df['predicted_num_unq'] = preds[:, 0], preds[:, 1]

In [ ]:
compare_df.head()

In [ ]:
# compare_df['predicted_cost+1M'] = DATA_TRANSFORMER.create_cost_column(compare_df['predicted_num_unq'], compare_df['predicted_total_secs'])

In [ ]:
compare_df.head()

In [ ]:
# for col in target_cols:
#     compare_df[col] = y_test[col]

# print('Total secs predicted score: ', show_scores_2(compare_df['total_secs+1M'], compare_df['predicted_total_secs']))
# print('Num unq predicted score: ',show_scores_2(compare_df['num_unq+1M'], compare_df['predicted_num_unq']))
# print('Cost predicted score: ', show_scores_2(X_test_cost, compare_df['predicted_cost+1M']))

# compare_df['cost+1M'] = X_test_cost

# compare_df['num_unq_diff'] = abs(compare_df['num_unq+1M'] - compare_df['predicted_num_unq'])
# compare_df['total_secs_diff'] = abs(compare_df['total_secs+1M'] - compare_df['predicted_total_secs'])
# compare_df['cost_diff'] = abs(X_test_cost - compare_df['predicted_cost+1M'])


# compare_df[['total_secs+1M', 'predicted_total_secs', 'total_secs_diff', 'num_unq+1M', 'predicted_num_unq', 'num_unq_diff', 'cost+1M', 'predicted_cost+1M', 'cost_diff']]

show_scores_2(y_test, preds)

In [ ]:
show_scores_2(y_test['cost+1M'], compare_df['predicted_cost'])

In [ ]:
preds

In [ ]:
preds